# **Perspectiva General de Mercado - DatAlpine**
*Filtro de tipo de mercado*

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
import numpy as np

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre and '2024' in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca']

In [2]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    df.rename(columns={'Categoria':'categoria','Category':'categoria','Propiedad':'propiedad','Precio':'precio','Metros_total':'metros_total','m_total':'metros_total','m_construido':'metros_construido','Metros_construido':'metros_construido','m2_contruido':'metros_construido','Estacionamientos':'estacionamiento','Recamaras':'recamaras','Banos':'baño','Medio_banos':'medio_baño','Banos_Total':'baño_total','Precio_M2_Construido':'precio_m2_construido','precio_m2_contruido':'precio_m2_construido'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [3]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id','categoria','propiedad','precio','metros_total','metros_construido', 'precio_m2_construido','estacionamiento','recamaras', 'baño', 'medio_baño', 'baño_total']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [4]:
df_total = pd.concat(dataframes_list, ignore_index=True)

In [5]:
df_total.columns

Index(['id', 'categoria', 'propiedad', 'precio', 'metros_total',
       'metros_construido', 'precio_m2_construido', 'estacionamiento',
       'recamaras', 'baño', 'medio_baño', 'baño_total'],
      dtype='object')

In [6]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11650 entries, 0 to 11649
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    11650 non-null  int64  
 1   categoria             11650 non-null  object 
 2   propiedad             11650 non-null  object 
 3   precio                11650 non-null  int64  
 4   metros_total          11650 non-null  int64  
 5   metros_construido     11650 non-null  int64  
 6   precio_m2_construido  11650 non-null  float64
 7   estacionamiento       11650 non-null  int64  
 8   recamaras             11650 non-null  int64  
 9   baño                  11650 non-null  int64  
 10  medio_baño            11650 non-null  int64  
 11  baño_total            11650 non-null  float64
dtypes: float64(2), int64(8), object(2)
memory usage: 1.1+ MB


In [7]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
#def asignar_categoria(precio):
#    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
#        if limite_inferior <= precio < limite_superior:
#            return categoria
#    return None  # En caso de que el precio no caiga en ningún rango (caso raro)
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return "Sin categoría"  # Asignar una etiqueta clara en lugar de None

# Convertir 'precio' a numérico y manejar errores
df_total['precio'] = pd.to_numeric(df_total['precio'], errors='coerce')
# Llenar valores faltantes con un número por defecto, por ejemplo, 0
df_total['precio'] = df_total['precio'].fillna(0).astype(int)
df_total = df_total[df_total['precio'] > 0]
df_total['precio'] = df_total['precio'].fillna(0)
df_total = df_total.drop(columns=['categoria'], errors='ignore')

df_total['categoria'] = df_total['precio'].apply(asignar_categoria)
categorias_invalidas = df_total[df_total['categoria'].isnull()]
if not categorias_invalidas.empty:
    print("Precios sin categoría:")
    print(categorias_invalidas[['precio']])


In [8]:
import numpy as np 
condition = (df_total['baño_total'] % 1 == 0.5) # Valores que contienen .5    
# Convertir al número entero anterior
df_total.loc[condition, 'baño_total'] = np.floor(df_total['baño_total']) 
df_total['baño_total'] = df_total['baño_total'].clip(lower=1,upper=8) # Limitar los valores 
df_total['estacionamiento'] = df_total['estacionamiento'].clip(lower=1,) #upper=8
df_total['recamaras'] = df_total['recamaras'].clip(lower=1,)#upper=8 

df_total = df_total[df_total['precio'] != 0]
df_total = df_total[df_total['precio'] > 4800]
df_total = df_total[df_total['metros_total'] > 20]
df_total = df_total[df_total['metros_construido'] > 20]

df_total = df_total[df_total['metros_construido'] > 20]


In [9]:
df_total['metros_construido'].sort_values().unique()

array([   30,    36,    37,    38,    40,    41,    43,    44,    45,
          46,    47,    48,    49,    50,    51,    52,    53,    54,
          55,    56,    57,    58,    60,    61,    62,    63,    64,
          65,    66,    67,    68,    69,    70,    71,    72,    73,
          74,    75,    76,    77,    78,    79,    80,    81,    82,
          83,    84,    85,    86,    87,    88,    89,    90,    91,
          92,    93,    94,    95,    96,    97,    98,    99,   100,
         101,   102,   103,   104,   105,   106,   107,   108,   109,
         110,   111,   112,   113,   114,   115,   116,   117,   118,
         119,   120,   121,   122,   123,   124,   125,   126,   127,
         128,   129,   130,   131,   132,   133,   134,   135,   136,
         137,   138,   139,   140,   141,   142,   143,   144,   145,
         146,   147,   148,   149,   150,   151,   152,   153,   154,
         155,   156,   157,   158,   159,   160,   161,   162,   163,
         164,   165,

- Casa promedio
- Casa Mediana
- Precio mercado

- m2 Terreno
- M2 Construcción
- Precio m2 construcción

# Casa promedio

In [10]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_promedios = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
grupo_promedios.astype(int)
# Renombrar las columnas para mayor claridad
grupo_promedios.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)

# Calcular el promedio de estacionamientos, recámaras y baños en general (sin segmento específico)
promedios_generales = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].mean(),0)
promedios_generales = promedios_generales.astype(int)
# Renombrar las columnas para mayor claridad
promedios_generales.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
# Agregar los promedios generales al dataframe de grupo_promedios
promedios_generales = pd.DataFrame(promedios_generales).T
promedios_generales['segmento'] = 'general'

casa_promedio = pd.concat([grupo_promedios, promedios_generales.set_index('segmento')], ignore_index=False)



print(grupo_promedios)
print(promedios_generales)
print(casa_promedio)


# Convertir el dataframe a JSON
grupo_promedios_json = casa_promedio.to_json(orient='index')

# Guardar el JSON en un archivo
with open('assets/tablas/grupo_casa_promedio.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      3.0        4.0    3.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        4.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    2.0
E                      2.0        3.0    2.0
L                      3.0        4.0    3.0
S                      2.0        3.0    2.0
general                2.0        3.0    2.0


# Casa Mediana

In [11]:
df_total['segmento'] = df_total['categoria'].str[0]
grupo_mediana = round(df_total.groupby('segmento')[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
grupo_mediana.astype(int)
# Renombrar las columnas para mayor claridad
grupo_mediana.rename(columns={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
print(grupo_mediana)

# Calcular mediana de estacionamientos, recámaras y baños en general (sin segmento específico)
mediana_general = round(df_total[['estacionamiento', 'recamaras', 'baño_total']].median(),0)
mediana_general = mediana_general.astype(int)
# Renombrar las columnas para mayor claridad
mediana_general.rename(index={
    'estacionamiento': 'Estacionamientos',
    'recamaras': 'Recámaras',
    'baño_total': 'Baños',
}, inplace=True)
mediana_general = pd.DataFrame(mediana_general).T
mediana_general['segmento'] = 'general'
print(mediana_general)

casa_media = pd.concat([grupo_mediana, mediana_general.set_index('segmento')], ignore_index=False)
print(casa_media)

# Guardar el JSON en un archivo
grupo_promedios_json = casa_media.to_json(orient='index')
with open('assets/tablas/grupo_casa_mediana.json', 'w') as f:
    f.write(grupo_promedios_json)

          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    1.0
E                      1.0        3.0    1.0
L                      2.0        4.0    3.0
S                      2.0        3.0    2.0
   Estacionamientos  Recámaras  Baños segmento
0                 2          3      2  general
          Estacionamientos  Recámaras  Baños
segmento                                    
A                      2.0        3.0    2.0
B                      2.0        3.0    2.0
C                      2.0        3.0    2.0
D                      2.0        3.0    1.0
E                      1.0        3.0    1.0
L                      2.0        4.0    3.0
S                      2.0        3.0    2.0
general                2.0        3.0    2.0


# Precio Mercado

In [12]:
grupo_descripcion = df_total.groupby('segmento')['precio'].describe().reset_index()
grupo_descripcion = grupo_descripcion[['segmento','min', 'max','mean','50%']] #Columnas deseadas
grupo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)
grupo_descripcion 

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,3255000.0,3999000.0,3.601984e+06,3600000.0
1,B,2500002.0,3242320.0,2.881962e+06,2890000.0
2,C,1750050.0,2499000.0,2.115324e+06,2140000.0
3,D,1005000.0,1749605.0,1.396118e+06,1400000.0
4,E,55000.0,90000000.0,1.733755e+06,670000.0
5,L,12100000.0,21000000.0,1.618773e+07,15500000.0
6,S,750000.0,22000000.0,5.029258e+06,4500000.0


In [13]:
# Función para formatear valores en formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
grupo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17660\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,255,000.00
1     $2,500,002.00
2     $1,750,050.00
3     $1,005,000.00
4        $55,000.00
5    $12,100,000.00
6       $750,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descripcion.iloc[:, i] = grupo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17660\913539489.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $3,999,000.00
1     $3,242,320.00
2     $2,499,000.00
3     $1,749,605.00
4    $90,000,000.00
5    $21,000,000.00
6    $22,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_descrip

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,255,000.00","$3,999,000.00","$3,601,983.91","$3,600,000.00"
1,B,"$2,500,002.00","$3,242,320.00","$2,881,961.62","$2,890,000.00"
2,C,"$1,750,050.00","$2,499,000.00","$2,115,324.05","$2,140,000.00"
3,D,"$1,005,000.00","$1,749,605.00","$1,396,118.36","$1,400,000.00"
4,E,"$55,000.00","$90,000,000.00","$1,733,755.37","$670,000.00"
5,L,"$12,100,000.00","$21,000,000.00","$16,187,727.27","$15,500,000.00"
6,S,"$750,000.00","$22,000,000.00","$5,029,258.22","$4,500,000.00"


In [14]:
todo_descripcion = df_total['precio'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_17660\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $55,000.00
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17660\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio    $90,000,000.00
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_17660\2840734770.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will rais

,Mínimo,Máximo,Promedio,Mediana,segmento
precio,"$55,000.00","$90,000,000.00","$3,216,403.68","$2,400,000.00",general


In [15]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_mercado.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [16]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$3,255,000.00","$3,999,000.00","$3,601,983.91","$3,600,000.00"
1,B,"$2,500,002.00","$3,242,320.00","$2,881,961.62","$2,890,000.00"
2,C,"$1,750,050.00","$2,499,000.00","$2,115,324.05","$2,140,000.00"
3,D,"$1,005,000.00","$1,749,605.00","$1,396,118.36","$1,400,000.00"
4,E,"$55,000.00","$90,000,000.00","$1,733,755.37","$670,000.00"
5,L,"$12,100,000.00","$21,000,000.00","$16,187,727.27","$15,500,000.00"
6,S,"$750,000.00","$22,000,000.00","$5,029,258.22","$4,500,000.00"
7,general,"$55,000.00","$90,000,000.00","$3,216,403.68","$2,400,000.00"


In [17]:
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]

# Función para limpiar valores con unidades y convertirlos a float
def limpiar_valor(valor):
    # Eliminar unidades (m², $, etc.) y convertir a float
    if isinstance(valor, str):
        return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))
    else:
        return valor

# Aplicar limpieza de valores en las columnas que contienen medidas
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)
# Función para formatear nuevamente los valores en el formato deseado
def formato_monetario(valor):
    return f"${valor:,.2f}"  # Formato con $ y 2 decimales, separador de miles

# Volver a aplicar el formato monetario después de los ajustes
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)



In [95]:
# Limpiar valores y convertir a float
def limpiar_valor(valor):
    return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)

# Verificar el orden de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]
resultados_finales_descripcion = resultados_finales_descripcion.set_index('segmento').reindex(orden_segmentos).reset_index()

# Validar que los valores no sean inconsistentes
for i in range(1, len(resultados_finales_descripcion)):
    for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
        valor_actual = resultados_finales_descripcion.loc[i, columna]
        valor_anterior = resultados_finales_descripcion.loc[i - 1, columna]
        
        # Ajustar valores inconsistentes
        if valor_actual < valor_anterior:
            resultados_finales_descripcion.loc[i, columna] = valor_anterior

# Volver a formatear los valores a formato monetario
def formato_monetario(valor):
    return f"${valor:,.2f}"

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)

# Guardar los resultados en un archivo JSON
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='records', force_ascii=False)
with open('assets/tablas/grupo_precio_mercado_ajustado.json', 'w', encoding='utf-8') as f:
    f.write(resultados_finales_desc_json)


In [96]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,E,"$55,000.00","$90,000,000.00","$1,733,755.37","$670,000.00"
1,D,"$1,005,000.00","$90,000,000.00","$1,733,755.37","$1,400,000.00"
2,C,"$1,750,050.00","$90,000,000.00","$2,115,324.05","$2,140,000.00"
3,B,"$2,500,002.00","$90,000,000.00","$2,881,961.62","$2,890,000.00"
4,A,"$3,255,000.00","$90,000,000.00","$3,601,983.91","$3,600,000.00"
5,S,"$3,255,000.00","$90,000,000.00","$5,029,258.22","$4,500,000.00"
6,L,"$12,100,000.00","$90,000,000.00","$16,187,727.27","$15,500,000.00"
7,ELITE,$nan,$nan,$nan,$nan
8,general,"$55,000.00","$90,000,000.00","$3,216,403.68","$2,400,000.00"


In [18]:
# Especificar el orden de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE", "general"]

# Limpiar valores y convertir a float
def limpiar_valor(valor):
    if isinstance(valor, str):
        return float(valor.replace(" m²", "").replace("$", "").replace(",", ""))
    else:
        return valor

# Aplicar limpieza de valores en las columnas que contienen medidas
for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(limpiar_valor)

# Calcular el "general" basado en los resultados de los segmentos
# Excluir el "general" de los datos de los segmentos para hacer el cálculo
segmentos_sin_general = resultados_finales_descripcion[resultados_finales_descripcion['segmento'] != 'general']

# Calcular los valores agregados para "general"
general_data = segmentos_sin_general[['Mínimo', 'Máximo', 'Promedio', 'Mediana']].agg(['min', 'max', 'mean', 'median']).T
general_data['segmento'] = 'general'

# Renombrar las columnas para que se ajusten al formato deseado
general_data.rename(columns={
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
    'median': 'Mediana',
}, inplace=True)

# Concatenar el "general" con la tabla de segmentos
resultados_finales_descripcion = pd.concat([segmentos_sin_general, general_data], ignore_index=True)

# Reordenar los resultados finales según el orden especificado
resultados_finales_descripcion['segmento'] = pd.Categorical(resultados_finales_descripcion['segmento'], categories=orden_segmentos, ordered=True)
resultados_finales_descripcion = resultados_finales_descripcion.sort_values('segmento').reset_index(drop=True)

# Formatear los valores monetarios
def formato_monetario(valor):
    return f"${valor:,.2f}"

for columna in ["Mínimo", "Máximo", "Promedio", "Mediana"]:
    resultados_finales_descripcion[columna] = resultados_finales_descripcion[columna].apply(formato_monetario)

# Guardar el JSON con los resultados finales
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='records', force_ascii=False)
with open('assets/tablas/grupo_precio_mercado_ordenado.json', 'w', encoding='utf-8') as f:
    f.write(resultados_finales_desc_json)

# Mostrar los resultados finales
resultados_finales_descripcion


,segmento,Mínimo,Máximo,Promedio,Mediana
0,E,"$55,000.00","$90,000,000.00","$1,733,755.37","$670,000.00"
1,D,"$1,005,000.00","$1,749,605.00","$1,396,118.36","$1,400,000.00"
2,C,"$1,750,050.00","$2,499,000.00","$2,115,324.05","$2,140,000.00"
3,B,"$2,500,002.00","$3,242,320.00","$2,881,961.62","$2,890,000.00"
4,A,"$3,255,000.00","$3,999,000.00","$3,601,983.91","$3,600,000.00"
5,S,"$750,000.00","$22,000,000.00","$5,029,258.22","$4,500,000.00"
6,L,"$12,100,000.00","$21,000,000.00","$16,187,727.27","$15,500,000.00"
7,general,"$55,000.00","$12,100,000.00","$3,059,293.14","$1,750,050.00"
8,general,"$1,749,605.00","$90,000,000.00","$20,641,417.86","$3,999,000.00"
9,general,"$1,396,118.36","$16,187,727.27","$4,706,589.83","$2,881,961.62"


# M2 Terreno

In [54]:
#df_total = df_total[(df_total['metros_construido'] > 40) 
#                    & ((df_total['segmento'].eq('C') & (df_total['metros_construido'] >= 60)) | (df_total['segmento'].eq('D') & (df_total['metros_construido'] >= 60))) 
#                       | (df_total['segmento'].isin(['A', 'B', 'E', 'S', 'L'])) 
#                       | (df_total['segmento'].eq('D') & (df_total['metros_construido'] > 60)))]

df_total = df_total[(df_total['metros_construido'] > 40]

df_total_terreno = df_total[df_total['metros_total'] != 0]
df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1) & (df_total_terreno['metros_total'] > 20)]

df_descripcion = df_total_terreno.groupby('segmento')['metros_total'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10560\447048446.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     75.00 m²
1     62.00 m²
2     44.00 m²
3     45.00 m²
4     38.00 m²
5    245.00 m²
6     66.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10560\447048446.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     1,077.00 m²
1    15,794.00 m²
2    14,079.00 m²
3     8,315.00 m²
4    10,000.00 m²
5     2,451.00 m²
6    32,240.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).a

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,75.00 m²,"1,077.00 m²",163.25 m²,144.00 m²
1,B,62.00 m²,"15,794.00 m²",171.46 m²,129.00 m²
2,C,44.00 m²,"14,079.00 m²",149.22 m²,120.00 m²
3,D,45.00 m²,"8,315.00 m²",122.10 m²,105.00 m²
4,E,38.00 m²,"10,000.00 m²",174.21 m²,105.00 m²
5,L,245.00 m²,"2,451.00 m²",666.88 m²,500.00 m²
6,S,66.00 m²,"32,240.00 m²",241.97 m²,168.00 m²


In [55]:
df_total_terreno = df_total[df_total['metros_total'] != 0]
df_total_terreno = df_total_terreno[(df_total_terreno['metros_total'] != 1)&(df_total_terreno['metros_total'] > 20)]

todo_descripcion = df_total_terreno[df_total_terreno['metros_total'] != 0]['metros_total'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_10560\2712485081.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    38.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10560\2712485081.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_total    32,240.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_10560\2712485081.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will ra

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_total,38.00 m²,"32,240.00 m²",186.87 m²,130.00 m²,general


In [56]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_terreno.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [57]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,75.00 m²,"1,077.00 m²",163.25 m²,144.00 m²
1,B,62.00 m²,"15,794.00 m²",171.46 m²,129.00 m²
2,C,44.00 m²,"14,079.00 m²",149.22 m²,120.00 m²
3,D,45.00 m²,"8,315.00 m²",122.10 m²,105.00 m²
4,E,38.00 m²,"10,000.00 m²",174.21 m²,105.00 m²
5,L,245.00 m²,"2,451.00 m²",666.88 m²,500.00 m²
6,S,66.00 m²,"32,240.00 m²",241.97 m²,168.00 m²
7,general,38.00 m²,"32,240.00 m²",186.87 m²,130.00 m²


# M2 Construcción

In [49]:
df_total_terreno = df_total[(df_total['metros_construido'] > 40) & (~df_total['metros_construido'].isin([0, 1,2,3]))]
df_descripcion = df_total_terreno.groupby('segmento')['metros_construido'].describe().reset_index()
df_descripcion = df_descripcion[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
df_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)


# Función para formatear valores en formato metros
def formato_metros(valor):
    return f"{valor:,.2f} m²"

# Aplicar formato en m2 a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
df_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7752\3111444512.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     75.00 m²
1     62.00 m²
2     44.00 m²
3     45.00 m²
4     41.00 m²
5    245.00 m²
6     85.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7752\3111444512.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       460.00 m²
1    29,743.00 m²
2       640.00 m²
3       472.00 m²
4    10,000.00 m²
5     1,000.00 m²
6     1,465.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_descripcion.iloc[:, i] = df_descripcion.iloc[:, i].astype(float).a

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,75.00 m²,460.00 m²,216.10 m²,222.00 m²
1,B,62.00 m²,"29,743.00 m²",230.52 m²,200.00 m²
2,C,44.00 m²,640.00 m²,159.53 m²,150.00 m²
3,D,45.00 m²,472.00 m²,119.73 m²,114.00 m²
4,E,41.00 m²,"10,000.00 m²",191.74 m²,120.00 m²
5,L,245.00 m²,"1,000.00 m²",567.30 m²,539.00 m²
6,S,85.00 m²,"1,465.00 m²",301.34 m²,280.00 m²


In [50]:
df_total_terreno = df_total[df_total['metros_construido'] > 40]

todo_descripcion = df_total_terreno['metros_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_7752\2628762141.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    41.00 m²
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7752\2628762141.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'metros_construido    29,743.00 m²
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_metros)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_7752\2628762141.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and 

,Mínimo,Máximo,Promedio,Mediana,segmento
metros_construido,41.00 m²,"29,743.00 m²",208.26 m²,170.00 m²,general


In [51]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([df_descripcion, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [52]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,75.00 m²,460.00 m²,216.10 m²,222.00 m²
1,B,62.00 m²,"29,743.00 m²",230.52 m²,200.00 m²
2,C,44.00 m²,640.00 m²,159.53 m²,150.00 m²
3,D,45.00 m²,472.00 m²,119.73 m²,114.00 m²
4,E,41.00 m²,"10,000.00 m²",191.74 m²,120.00 m²
5,L,245.00 m²,"1,000.00 m²",567.30 m²,539.00 m²
6,S,85.00 m²,"1,465.00 m²",301.34 m²,280.00 m²
7,general,41.00 m²,"29,743.00 m²",208.26 m²,170.00 m²


# Pecio M2 Construcción

In [37]:
df_total_terreno = df_total[~df_total['metros_construido'].isin([0, 1,2,3]) & (df_total['precio_m2_construido'] > 1000)]

grupo_filtrado = df_total_terreno.groupby('segmento')['precio_m2_construido'].describe().reset_index()
# Mantener solo las columnas que deseas (en este caso, min y max)
grupo_filtrado = grupo_filtrado[['segmento', 'min', 'max', 'mean', '50%']] #Columnas deseadas
grupo_filtrado.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores, excluyendo la columna 'segmento'
for i in range(1, 5):  # 1, 2, 3, 4 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)

grupo_filtrado 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_8632\2945803441.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $8,043.48
1     $2,674.28
2     $3,358.64
3     $2,400.66
4     $1,108.33
5    $17,375.00
6     $2,779.52
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_8632\2945803441.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0     $47,500.00
1     $44,692.00
2     $42,771.66
3     $38,000.00
4     $91,666.67
5     $61,224.49
6    $238,333.33
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  grupo_filtrado.iloc[:, i] = grupo_filtrado.iloc[:, i].astype(float

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$8,043.48","$47,500.00","$18,091.71","$16,237.29"
1,B,"$2,674.28","$44,692.00","$16,119.04","$14,950.00"
2,C,"$3,358.64","$42,771.66","$14,335.25","$14,000.01"
3,D,"$2,400.66","$38,000.00","$12,598.79","$12,234.04"
4,E,"$1,108.33","$91,666.67","$8,194.75","$6,729.41"
5,L,"$17,375.00","$61,224.49","$30,200.68","$27,777.78"
6,S,"$2,779.52","$238,333.33","$21,695.69","$19,218.80"


In [41]:
todo_descripcion = df_total[(~df_total['metros_construido'].isin([0, 1,2,3])) & (df_total['precio_m2_construido'] > 1000)]['precio_m2_construido'].describe()[['min', 'max', 'mean', '50%']].to_frame().T
todo_descripcion['segmento'] = 'general'

todo_descripcion.rename(columns={
    '50%': 'Mediana',
    'min': 'Mínimo',
    'max': 'Máximo',
    'mean': 'Promedio',
}, inplace=True)

# Aplicar formato monetario a los valores
for i in range(4):  # 0, 1, 2, 3 corresponden a 'Mínimo', 'Máximo', 'Promedio', 'Mediana'
    todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)

todo_descripcion 

C:\Users\yoe11\AppData\Local\Temp\ipykernel_8632\378671069.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $1,108.33
Name: Mínimo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_8632\378671069.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'precio_m2_construido    $238,333.33
Name: Máximo, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  todo_descripcion.iloc[:, i] = todo_descripcion.iloc[:, i].astype(float).apply(formato_monetario)
C:\Users\yoe11\AppData\Local\Temp\ipykernel_8632\378671069.py:13: FutureWarning: Setting an item of incompatible dtype is deprec

,Mínimo,Máximo,Promedio,Mediana,segmento
precio_m2_construido,"$1,108.33","$238,333.33","$15,454.06","$14,444.44",general


In [42]:
# Guardar el JSON en un archivo
resultados_finales_descripcion = pd.concat([grupo_filtrado, todo_descripcion], ignore_index=True)
resultados_finales_desc_json = resultados_finales_descripcion.to_json(orient='index')
with open('assets/tablas/grupo_precio_construidos.json', 'w') as f:
    f.write(resultados_finales_desc_json)

In [43]:
resultados_finales_descripcion

,segmento,Mínimo,Máximo,Promedio,Mediana
0,A,"$8,043.48","$47,500.00","$18,091.71","$16,237.29"
1,B,"$2,674.28","$44,692.00","$16,119.04","$14,950.00"
2,C,"$3,358.64","$42,771.66","$14,335.25","$14,000.01"
3,D,"$2,400.66","$38,000.00","$12,598.79","$12,234.04"
4,E,"$1,108.33","$91,666.67","$8,194.75","$6,729.41"
5,L,"$17,375.00","$61,224.49","$30,200.68","$27,777.78"
6,S,"$2,779.52","$238,333.33","$21,695.69","$19,218.80"
7,general,"$1,108.33","$238,333.33","$15,454.06","$14,444.44"
